# 03 · Pipeline Principal: Registro y Medición
**Objetivo:** emparejar 3 imágenes reales, estimar homografías (RANSAC), crear mosaico con *feather blending*, fijar escala con referencia conocida y medir elementos.

In [1]:
import os

# Ir al root del repo (sube un nivel desde /notebooks)
if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")

import glob
import numpy as np
import pandas as pd
import cv2 as cv
import matplotlib.pyplot as plt

from src.feature_detection import detect_and_describe
from src.matching import match_descriptors
from src.registration import estimate_homography, warp_images, feather_blend
from src.measurement import set_scale_by_two_points, measure_distance, interactive_pick_points
from src.utils import imread_color

plt.rcParams['figure.figsize'] = (10, 7)
plt.rcParams['axes.grid'] = True

DATA_DIR = "data/original"
FIG_DIR = "results/figures"
MEAS_DIR = "results/measurements"
os.makedirs(FIG_DIR, exist_ok=True)
os.makedirs(MEAS_DIR, exist_ok=True)

## 1) Carga de 3 imágenes

In [2]:
paths = sorted(glob.glob(os.path.join(DATA_DIR, '*')))
assert len(paths) >= 3, "Coloca al menos 3 imágenes en data/original"
I0, I1, I2 = [imread_color(p) for p in paths[:3]]
for i, p in enumerate(paths[:3]):
    print(f"[{i}] {os.path.basename(p)} -> {I0.shape if i==0 else (I1.shape if i==1 else I2.shape)}")

[0] IMG01.jpg -> (4032, 3024, 3)
[1] IMG02.jpg -> (4032, 3024, 3)
[2] IMG03.jpg -> (3024, 4032, 3)


## 2) Detección y *matching*
Usaremos **SIFT** por robustez (requiere `opencv-contrib-python`).

In [3]:
k0, d0 = detect_and_describe(I0, method="SIFT")
k1, d1 = detect_and_describe(I1, method="SIFT")
k2, d2 = detect_and_describe(I2, method="SIFT")

m10 = match_descriptors(d1, d0, strategy="FLANN", ratio=0.75)
m20 = match_descriptors(d2, d0, strategy="FLANN", ratio=0.75)

print("Matches I1->I0:", len(m10), " | Matches I2->I0:", len(m20))

Matches I1->I0: 5916  | Matches I2->I0: 3132


## 3) Estimación de homografías (RANSAC)

In [4]:
H10, in10 = estimate_homography(k1, k0, m10, ransac_thresh=3.0)
H20, in20 = estimate_homography(k2, k0, m20, ransac_thresh=3.0)
print("Inliers I1->I0:", int(in10.sum()), " | Inliers I2->I0:", int(in20.sum()))

Inliers I1->I0: 3552  | Inliers I2->I0: 1597


## 4) Warping y *feather blending*

In [5]:
# Deformar I1 e I2 al marco de I0
warped1, mask0 = warp_images(I1, I0, H10)
warped2, _ = warp_images(I2, I0, H20)

mosaico = feather_blend(warped1, warped2, mask0)
out_mosaic = os.path.join(FIG_DIR, "mosaico.png")
cv.imwrite(out_mosaic, mosaico)
print("Guardado mosaico:", out_mosaic)

plt.imshow(cv.cvtColor(mosaico, cv.COLOR_BGR2RGB)); plt.title("Mosaico"); plt.axis('off'); plt.show()

error: OpenCV(4.12.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\arithm.cpp:214: error: (-209:Sizes of input arguments do not match) The operation is neither 'array op array' (where arrays have the same size and type), nor 'array op scalar', nor 'scalar op array' in function 'cv::binary_op'


## 5) Fijar escala y medir
Para la escala, selecciona **dos puntos** sobre una referencia conocida (por ejemplo, altura del cuadro = **117 cm**).
Luego, valida midiendo el **ancho de la mesa** (valor conocido **161.1 cm**).
Después, mide **al menos 3 elementos adicionales** y guarda resultados.

In [ ]:
# --- Escala con referencia conocida ---
print("Selecciona 2 puntos de la referencia (ej. altura del cuadro = 117 cm)")
ref_pts = interactive_pick_points(mosaico, npoints=2)
scale = set_scale_by_two_points(ref_pts[0], ref_pts[1], real_distance_cm=117.0)

# --- Validación con mesa ---
print("Selecciona 2 puntos que representen el ancho de la mesa (valor real: 161.1 cm)")
mesa_pts = interactive_pick_points(mosaico, npoints=2)
mesa_cm = measure_distance(mesa_pts[0], mesa_pts[1], scale)
mesa_err = mesa_cm - 161.1
print(f"Ancho mesa estimado: {mesa_cm:.2f} cm | Error: {mesa_err:.2f} cm")

# --- Medir ≥3 elementos adicionales ---
mediciones = []
def medir_elemento(nombre: str):
    print(f"Selecciona 2 puntos para '{nombre}'")
    pts = interactive_pick_points(mosaico, npoints=2)
    d = measure_distance(pts[0], pts[1], scale)
    mediciones.append({"elemento": nombre, "dist_cm": d})
    print(f"{nombre}: {d:.2f} cm")

items = ["Elemento_1", "Elemento_2", "Elemento_3"]
for it in items:
    medir_elemento(it)

# Guardar a CSV
now = pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")
csv_path = os.path.join(MEAS_DIR, f"mediciones_{now}.csv")
df = pd.DataFrame(mediciones + [
    {"elemento": "VALIDACION_MESA", "dist_cm": mesa_cm, "error_vs_161_1": mesa_err}
])
df.to_csv(csv_path, index=False)
print("Guardado:", csv_path)